Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel $\rightarrow$ Restart) and then **run all cells** (in the menubar, select Cell $\rightarrow$ Run All).

Make sure you fill in any place that says `???`, `YOUR CODE HERE`, "???", "YOUR ANSWER HERE", as well as your name and collaborators below:

In [ ]:
val NAME = ""
val COLLABORATORS = ""

---

# Exercise: Syntax

<!-- 3 Expressions -->

<!-- 4 Binding and Scope -->

<!-- 8 Recursion -->

<!-- 9 Inductive Data Types -->

<!-- 11 Concrete Syntax -->

<!-- 12 Abstract Syntax and Parsing -->

<!-- 13 Exercise: Syntax -->

<!-- 14 Static Scoping -->

### Learning Goals

The primary learning goals of this assignment are to build intuition for
the following:

-   working with abstract syntax trees;
-   how grammars are used to specify the syntax of programming
    languages; and
-   the distinction between concrete and abstract syntax.

### Instructions

This assignment asks you to write Scala code. There are restrictions
associated with how you can solve these problems. Please pay careful
heed to those. If you are unsure, ask the course staff.

Note that `???` indicates that there is a missing function or code
fragment that needs to be filled in. Make sure that you remove the `???`
and replace it with the answer.

Use the test cases provided to test your implementations. You are also
encouraged to write your own test cases to help debug your work.
However, please delete any extra cells you may have created lest they
break an autograder.

### Imports

In [ ]:
// Run this cell FIRST before testing.
import $ivy.`org.scalatest::scalatest:3.2.19`, org.scalatest._, events._, flatspec._
def report(suite: Suite): Unit = suite.execute(stats = true)
def assertPassed(suite: Suite): Unit =
  suite.run(None, Args(new Reporter {
    def apply(e: Event) = e match {
      case e @ (_: TestFailed) => assert(false, s"${e.message} (${e.testName})")
      case _ => ()
    }
  }))
def passed(points: Int): Unit = {
  require(points >=0)
  if (points == 1) println("*** 🎉 Tests Passed (1 point) ***")
  else println(s"*** 🎉 Tests Passed ($points points) ***")
}
def test(suite: Suite, points: Int): Unit = {
  report(suite)
  assertPassed(suite)
  passed(points)
}

In [ ]:
// Run this cell FIRST before building your parser.
import $ivy.`org.scala-lang.modules::scala-parser-combinators:2.4.0`

## Abstract Syntax Trees

Let us consider the (abstract) syntax of a language of Boolean
expressions:

$$\begin{array}{lrrl}
  \text{Boolean expressions} & e& \mathrel{::=}& x
  \mid\neg e_1
  \mid e_1 \wedge e_2
  \mid e_1 \vee e_2 \\
  \text{variables} & x
\end{array}$$

### Defining an Inductive Data Type

We can write explicitly the above grammar with abstract syntax tree
nodes:

$$\begin{array}{rrrl}
\text{Boolean expressions $\texttt{BExpr}$} & e& \mathrel{::=}&
  \texttt{Var(} x\texttt{)} \\
  & & \mid& \texttt{Not(}e_1\texttt{)} \\
  & & \mid& \texttt{And(}e_1\texttt{,}\;e_2\texttt{)} \\
  & & \mid& \texttt{Or(}e_1\texttt{,}\;e_2\texttt{)} \\
\end{array}$$

<span class="theorem-title">**Exercise 1 (5 points)**</span> Define an
inductive data type `BExpr` in Scala following the above grammar. Use
the the names given above for the constructors, and use the Scala type
`String` to represent variable names.

**Edit this cell:**

In [ ]:
sealed trait BExpr
???

#### Tests

In [ ]:
val a: BExpr = Var("A")
val b: BExpr = Var("B")
val c: BExpr = Var("C")
val not_b: BExpr = Not(b)
val not_a: BExpr = Not(a)
val e1: BExpr = And(a,not_b)
val e2: BExpr = And(b, not_a)
val e3: BExpr = Or(e1,e2)
passed(5)

### Converting to Negation Normal Form

<span class="theorem-title">**Exercise 2 (10 points)**</span> Write a
function `nnf` to convert Boolean formulas from the above grammar to
their Negation Normal Form (NNF). A Boolean formula is said to be in NNF
iff the negation operator $\neg$ (i.e., $\texttt{Not}$) is only applied
to the variables. For example, the following formula is in NNF because
negation is only applied to $A$ or $B$, or both of which are variables:

$$
(A \wedge \neg B) \vee (B \land \neg A) \;.
$$

On the other hand, the following formula is not in NNF as negation is
applied to a complex expression:

$$
\neg(A \vee \neg B) \wedge (\neg B \vee C) \;.
$$

The negation normal form of the above formula is as follows:

$$
(\neg A \wedge \neg B) \wedge (\neg B \vee C) \;.
$$

**Edit this cell:**

In [ ]:
def nnf(e: BExpr): BExpr =
  // Hint: There are 7 cases.
  ???

#### Notes

In general, a formula can be converted to NNF by applying three rules:

Rule 1  
Double negation is cancelled: $\neg(\neg e) = e$ for any Boolean formula
$e$.

Rule 2  
De Morgan’s Law for conjunction:
$\neg(e_1 \wedge e_2) = \neg e_1 \vee \neg e_2$ for any two Boolean
formulas $e_1$ and $e_2$.

Rule 3  
De Morgan’s Law for disjunction:
$\neg(e_1 \vee e_2) = \neg e_1 \wedge \neg e_2$ for any two Boolean
formulas $e_1$ and $e_2$.

The `nnf` function will have a case for each of the above rule. In
addition, the function will also need to handle the following cases:

-   A variable $x$ or its negation $\neg x$ is already in NNF.
-   An expression $e_1 \wedge e_2$ is in NNF iff $e_1$ and $e_2$ are in
    NNF.
-   An expression $e_1 \vee e_2$ is in NNF iff $e_1$ and $e_2$ are in
    NNF.
-   An expression $\neg e_1$ is in NNF iff $e_1$ is in NNF and none of
    the above 3 rules apply.

If you handle all the 7 cases described above, then your `nnf` function
will likely be correct, though of course, you will want to test it.

#### Tests

In [ ]:
test(new AnyFlatSpec {
  val testcase1 = Var("A")
  val testcase2 = Not(Not(Var("B")))
  val testcase3 = And(Var("A"), Or(Var("B"), Var("C")))
  val testcase4 = Or(Var("A"), And(Var("B"), Var("C")))
  val testcase5 = Not(And(Var("A"), Var("B")))
  val testcase6 = Not(Or(Var("A"), Var("B")))
  val testcase7 = Or(And(testcase5,testcase6),testcase2)

  "nnf" should "Test Case 1" in { assertResult( Var("A") ) { nnf(testcase1)} }
  it should "Test Case 2" in { assertResult( Var("B") ) { nnf(testcase2) } }
  it should "Test Case 3" in { assertResult( And(Var("A"), Or(Var("B"), Var("C"))) ) { nnf(testcase3) } }
  it should "Test Case 4" in { assertResult( Or(Var("A"), And(Var("B"), Var("C"))) ) { nnf(testcase4) } }
  it should "Test Case 5" in { assertResult( Or(Not(Var("A")), Not(Var("B"))) ) { nnf(testcase5) } }
  it should "Test Case 6"  in { assertResult( And(Not(Var("A")), Not(Var("B"))) ) { nnf(testcase6) } }
  it should "Test Case 7" in { assertResult( Or(And(Or(Not(Var("A")), Not(Var("B"))), And(Not(Var("A")), Not(Var("B")))), Var("B")) ) { nnf(testcase7) } }
}, 10)

### Substitution

<span class="theorem-title">**Exercise 3 (5 points)**</span> Write a
function `subst` that substitutes *in* a given Boolean expression *with*
another expression *for* a given variable name. For example,
substituting in $$
\neg(A \vee B) \wedge (\neg B \vee C)
$$ with $D$ for $B$ yields $$
\neg(A \vee D) \wedge (\neg D \vee C) \;.
$$

**Edit this cell:**

In [ ]:
def subst(in_e: BExpr, with_e: BExpr, for_x: String): BExpr =
  ???

#### Tests

In [ ]:
test(new AnyFlatSpec {
  val testcase1 = Var("A")
  val testcase2 = Not(Var("B"))
  val testcase3 = Or(Var("A"), Var("B"))
  val testcase4 = And(Not(Var("C")), Or(Var("D"), Var("E")))
  val testcase5 = Or(And(Var("X"), Var("Y")), Or(Var("Z"), Var("X")))

  it should "Test Case 1" in { assertResult( Var("X") ) { subst(testcase1, Var("X"), "A") } }
  it should "Test Case 2" in { assertResult( Not(Var("Y")) ) { subst(testcase2, Var("Y"), "B") } }
  it should "Test Case 3" in { assertResult( Or(Var("C"), Var("B")) ) { subst(testcase3, Var("C"), "A") } }
  it should "Test Case 4" in { assertResult( And(Not(Var("C")), Or(Var("D"), Var("F"))) ) { subst(testcase4, Var("F"), "E") } }
  it should "Test Case 5" in { assertResult( Or(And(Var("W"), Var("Y")), Or(Var("Z"), Var("W"))) ) { subst(testcase5, Var("W"), "X") } }
}, 5)

## Concrete Syntax

### Precedence Detective

Consider the Scala binary expressions $$\begin{array}{rrrl}
  \text{expressions} & e& \mathrel{::=}& n\mid
  e_1 \mathrel{\texttt{-}} e_2 \mid
  e_1 \mathrel{\texttt{<<}} e_2 \\
  \text{integers} & n
\end{array}$$

<span class="theorem-title">**Exercise 4 (5 points)**</span> Write Scala
expressions to determine if $\texttt{-}$ has higher precedence than
$\texttt{<<}$ or vice versa. To do, write an expression bound to
`e_no_parens` that uses no parentheses. Then, bind to `e_higher_-` the
expression that adds parentheses to the `e_no_parens` expression
corresponding to the case if $\texttt{-}$ has higher precedence than
$\texttt{<<}$, and bind to `e_higher_<<` the expression adds parentheses
corresponding to the other case.

**Edit this cell:**

In [ ]:
val e_no_parens =
  ???

val e_higher_- =
  ???

val e_higher_<< =
  ???

Make sure that you are checking for precedence and not for left or right
associativity.

#### Assertions

In [ ]:
assert(e_no_parens == e_higher_- || e_no_parens == e_higher_<<)
assert(e_higher_- != e_higher_<<)
passed(5)

#### Explanation

<span class="theorem-title">**Exercise 5 (5 points)**</span> Explain how
you arrived at the relative precedence of $\texttt{-}$ and $\texttt{<<}$
based on the output that you saw in the Scala interpreter.

**Edit this cell:**

YOUR ANSWER HERE

## Parse Trees

Consider the following grammar:

<span id="eq-ite-grammar">$$\begin{array}{rrrl}
  \text{expressions} & e& \mathrel{::=}& x\mid e\;\texttt{?}\;e\;\texttt{:}\;e \\
  \text{variables} & x& \mathrel{::=}& \texttt{a} \mid\texttt{b}
\end{array} \qquad(1)$$</span>

Consider the following data type for representing parse trees:

In [ ]:
sealed trait ParseTree
case class Leaf(term: String) extends ParseTree
case class Node(nonterm: String, children: List[ParseTree]) extends ParseTree

Observe that a `ParseTree` is just an $n$-ary tree containing `String`s.
A `Leaf` is a terminal containing the lexemes (i.e., letters from the
alphabet), while a `Node` is represents a non-terminal with a `String`
for the name of the non-terminal a list of `children` `ParseTree`s.

For example, the following are `ParseTree`s for this grammar
(<a href="#eq-ite-grammar" class="quarto-xref">Equation 1</a>):

In [ ]:
val p1 = Node("x", Leaf("a") :: Nil)
val p2 = Node("e", Node("x", Leaf("a") :: Nil) :: Nil)

We provide a function that pretty-prints a `ParseTree` for *this
grammar* (<a href="#eq-ite-grammar" class="quarto-xref">Equation 1</a>).

In [ ]:
def pretty(t: ParseTree): Option[String] = {
  val alphabet = Set("a", "b", "?", ":")
  t match {
    // e ::= x
    case Node("e", (x @ Node("x", _)) :: Nil) => pretty(x)
    // e ::= e ? e : e
    case Node("e", (e1 @ Node("e", _)) :: Leaf("?") :: (e2 @ Node("e", _)) :: Leaf(":") :: (e3 @ Node("e", _)) :: Nil) =>
      for { s1 <- pretty(e1); s2 <- pretty(e2); s3 <- pretty(e3) }
      yield s"$s1 ? $s2 : $s3"
    // x ::= a
    case Node("x", Leaf("a") :: Nil) => Some("a")
    // x ::= b
    case Node("x", Leaf("b") :: Nil) => Some("b")
    // failure
    case _ => None
  }
}

pretty(p1)
pretty(p2)

Since it is possible to have `ParseTree`s that are not recognized by
this grammar, the `pretty` function has return type `Option[String]`.
Calling `pretty` on `ParseTree`s that are not in this grammar return
`None`:

In [ ]:
pretty(Leaf("a"))
pretty(Node("x", Leaf("c") :: Nil))
pretty(Node("y", Leaf("a") :: Nil))

Note that the `pretty` function makes use of Scala’s `for`-`yield`
expressions, which you do not need to understand for this exercise.

#### Exercise

<span class="theorem-title">**Exercise 6 (5 points)**</span> Give a
parse tree for the sentence in the grammar `a ? b : a`.

**Edit this cell:**

In [ ]:
val parseTree1: ParseTree =
  ???

#### Assertion

In [ ]:
assert(pretty(parseTree1) == Some("a ? b : a"))
passed(5)

#### Exercise

<span class="theorem-title">**Exercise 7 (10 points)**</span> Show that
this grammar
(<a href="#eq-ite-grammar" class="quarto-xref">Equation 1</a>) is
ambiguous by giving two parse trees for the sentence in the grammar
`a ? b : a ? b : a`.

**Edit this cell:**

In [ ]:
val parseTree2: ParseTree =
  ???

val parseTree3: ParseTree =
  ???

In [ ]:
pretty(parseTree2)
pretty(parseTree3)

#### Assertions

In [ ]:
assert(parseTree2 != parseTree3)
assert(pretty(parseTree2).isDefined)
assert(pretty(parseTree3).isDefined)
passed(5)

In [ ]:
assert(pretty(parseTree2) == Some("a ? b : a ? b : a"))
assert(pretty(parseTree3) == Some("a ? b : a ? b : a"))
passed(5)

## Defining Grammars

In this question, we define a BNF grammar for *floating-point numbers*
that are made up of a fraction (e.g., `5.6` or `3.123` or `-2.5`)
followed by an optional exponent (e.g., `E10` or `E-10`).

More precisely for this exercise, our floating-point numbers

-   [ ] must have a decimal point,
-   [ ] do not have leading zeros,
-   [ ] can have any number of trailing zeros,
-   [ ] non-zero exponents if it exists,
-   [ ] must have non-zero fraction to have an exponent, and
-   [ ] cannot have a ‘-’ in front of a zero number; also,
-   [ ] the exponent cannot have leading zeros.

The exponent, if it exists, is the letter `E` followed by an integer.
For example, the following are floating-point numbers: `0.0`, `3.5E3`,
`3.123E30`, `-2.5E2`, `-2.5E-2`, `3.50`, and `3.01E2`.

The following are examples of strings that are *not* floating-point
numbers by our definition: `0`, `-0.0`, `3.E3`, `E3`, `3.0E4.5`, and
`4E4`.

For this exercise, let us assume that the tokens are characters in the
following alphabet $\Sigma$:

$\Sigma \stackrel{\text{\tiny def}}{=}\{$ `0`, `1`, `2`, `3`, `4`, `5`,
`6`, `7`, `8`, `9`, `E`, `-`, `.` $\}$

<span class="theorem-title">**Exercise 8 (10 points)**</span> Translate
a grammar into a parser `FloatParser.float: Parser[String]` using the
Scala Parsing Combinator Library that recognizes the language of
floating-point numbers described above. Since we do not care about the
correctness of the grammar, we let parse result simply be the input
string.

We suggest that you first use the scratch cell below to give a grammar
in BNF notation. Your grammar should be completely defined using the
alphabet above as the terminals (i.e., it should not count on a
non-terminal that it does not itself define).

SCRATCH CELL

**Edit this cell:**

In [ ]:
object FloatParser extends scala.util.parsing.combinator.RegexParsers {
  val concat2: String ~ String => String = { case s1 ~ s2 => s1 + s2 }
  val concat3: String ~ String ~ String => String = { case s1 ~ s2 ~ s3 => s1 + s2 + s3 }
  val concat4: String ~ String ~ String ~ String => String = { case s1 ~ s2 ~ s3 ~ s4 => s1 + s2 + s3 + s4 }
  val concat5: String ~ String ~ String ~ String ~ String => String = { case s1 ~ s2 ~ s3 ~ s4 ~ s5 => s1 + s2 + s3 + s4 + s5 }

  def float: Parser[String] =
    ???

  // You may add additional "non-terminal" methods replacing this `???`
  ???

  def zeroOrMoreDigits: Parser[String] =
    digit ~ zeroOrMoreDigits ^^ concat2 |
    success("")
    
  def digit: Parser[String] = "0" | anyOneToNine

  def anyOneToNine: Parser[String] = "1" | "2" | "3" | "4" | "5" | "6" | "7" | "8" | "9"

  def sign: Parser[String] =
    "-" |
    success("")

  def parse(str: String): Option[String] = parseAll(float, str) match {
    case Success(str, _) => Some(str)
    case Failure(_, _) | Error(_, _) => None
  }
}

#### Notes

-   The `success[A](a: A): Parser[A]` method corresponds to an
    $\varepsilon$ production in BNF. It returns a parser that is
    successful without consuming any input yielding the parse result
    `a`. If you use it in your parser, make sure it is the *last*
    production for the non-terminal.
-   We provide some helper functions
    `concat2: String ~ String => String`,
    `concat3: String ~ String ~ String => String`, etc. that simply
    concatenate their input strings together. These helper functions are
    the only semantic actions you need.
-   We have provided some basic parsers `sign`, `anyOneToNine`, `digit`,
    `zeroOrMoreDigits` that you may use if you like.
-   You may edit the `parse` interface function that we have provided to
    start from a different non-terminal while you’re developing, but
    make sure it is `float` in the end and that your starting
    non-terminal is `float`. Alternatively, you may add additional such
    interface functions with a different name for your testing.

#### Tests

In [ ]:
test(new AnyFlatSpec {
  val tests_positive = List(
    "0.0",
    "3.5E3",
    "3.123E30",
    "-2.5E2",
    "-2.5E-2",
    "3.50",
    "3.01E2"
  )

  behavior of "FloatParser"

  for (t <- tests_positive) {
    it should s"parse $t" in {
      assertResult( Some(t) ) {
        FloatParser.parse(t)
      }
    }
  }

  val tests_negative = List(
    "0",
    "-0.0",
    "3.E3",
    "E3",
    "3.0E4.5",
    "4E4"
  )

  for (t <- tests_negative) {
    it should s"fail to parse $t" in {
      assertResult( None ) {
        FloatParser.parse(t)
      }
    }
  }
}, 10)